In [1]:
SANDBOX_NAME = # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"



# Ejemplo de Param Grid

Para encontrar los mejores hiperaparámetros para un modelo, se puede definir un conjunto de posibles valores para cada hiperparámetro, y crear un programa que entrene modelos con cada combinación posible de ellos, y almacene el mejor modelo dada una metrica. Además se puede mejorar junto con la técnica de Validación Cruzada.

In [4]:
# Respuesta

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

spark = SparkSession.builder.getOrCreate()



Primer paso, cargar algunos datos de prueba e inspeccionar.

In [5]:
# Respuesta

bank = spark.read.csv(DATA_PATH+'bank.csv', sep=';', header=True, inferSchema=True)

bank.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [22]:
# Respuesta

bank.show(5)

+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|        job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30| unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|   services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35| management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30| management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|blue-collar|married|secondary|     no|      0|    yes|  no| unknown|  5|  may|     22



Busquemos valores nulos en todas las columnas y descartemos filas que tengan nulos en ellas. Ya vimos anteriormente cómo trabajar con valores nulos.

In [7]:
# Respuesta

for column in bank.columns:
    print("Looking for nulls at " +column)
    num_nulls = bank.where(F.col(column).isNull()).count()
    if  num_nulls != 0:
        print("There are null values in the column {}".format(column))
        bank = bank.where(F.col(column).isNotNull())
        if num_nulls == 0:
            print("The column {} is free from null values".format(column))
    else:
        print("-> None null found.")

Looking for nulls at age
-> None null found.
Looking for nulls at job
-> None null found.
Looking for nulls at marital
-> None null found.
Looking for nulls at education
-> None null found.
Looking for nulls at default
-> None null found.
Looking for nulls at balance
-> None null found.
Looking for nulls at housing
-> None null found.
Looking for nulls at loan
-> None null found.
Looking for nulls at contact
-> None null found.
Looking for nulls at day
-> None null found.
Looking for nulls at month
-> None null found.
Looking for nulls at duration
-> None null found.
Looking for nulls at campaign
-> None null found.
Looking for nulls at pdays
-> None null found.
Looking for nulls at previous
-> None null found.
Looking for nulls at poutcome
-> None null found.
Looking for nulls at y
-> None null found.




Tras limpiar el dataset de nulos, podemos continuar preparando las variables y entrenando un modelo.
Para mantenerlo sencillo, apuntaremos las columnas binarias para evitar aplicarles onehot.

- Aplicamos el string indexer a todas las columnas tipo string (estamos asumiendo aquí que todas las columnas tipo string son categóricas)
- Aplicamos one hot encoder a todas las variables string no binarias
- Tras el string indexer Y el one hot encoder en las variables no binarias, removemos la columna resultado del string indexer para quedarnos sólo con la salida del one hot encoder. Y le cambiamos el nombre a la salida del one hot encoder a *_encoded*. Así sólo tendremos una variable *_encoded* para cada variable transformada en lugar de tener en el dataset el resultado del string indexer Y el del one hot encoder.

In [9]:
# Respuesta

binary_columns = ["default","housing","loan","y"]

string_columns = [item[0] for item in bank.dtypes if item[1].startswith('string')]

# making a copy of our dataset
bank_many_steps = bank.select("*")

for col in string_columns:
    print(col)
    string_indexer = StringIndexer(inputCol=col, outputCol=col+"_encoded")
    string_indexer_model = string_indexer.fit(bank_many_steps)
    bank_many_steps = string_indexer_model.transform(bank_many_steps)
    
    if col not in binary_columns:
        onehotencoder = OneHotEncoder(dropLast=False, inputCol= string_indexer.getOutputCol(), outputCol=col+"_encoded_tmp")
        
        bank_many_steps = onehotencoder.transform(bank_many_steps)
        bank_many_steps = bank_many_steps.drop(string_indexer.getOutputCol())
        bank_many_steps = bank_many_steps.withColumnRenamed(onehotencoder.getOutputCol(),string_indexer.getOutputCol())
    

job
marital
education
default
housing
loan
contact
month
poutcome
y


In [23]:
# Respuesta

bank_many_steps.show(5)

+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+--------------+---------------+-----------------+---------------+---------------+------------+---------------+--------------+----------------+---------+--------------------+
|age|        job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|   job_encoded|marital_encoded|education_encoded|default_encoded|housing_encoded|loan_encoded|contact_encoded| month_encoded|poutcome_encoded|y_encoded|  assembled_features|
+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+--------------+---------------+-----------------+---------------+---------------+------------+---------------+--------------+----------------+---------+--------------------+
| 30| unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79



Listamos columnas de entrenamiento y seleccionamos columna de target

In [11]:
# Respuesta

target_column = "y_encoded"
numeric_columns = [element[0] for element in bank_many_steps.dtypes if element[1] != 'string' not in element[0]]
columns_for_model = [c for c in numeric_columns if c!=target_column]

In [12]:
# Respuesta

columns_for_model

['age',
 'balance',
 'day',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'job_encoded',
 'marital_encoded',
 'education_encoded',
 'default_encoded',
 'housing_encoded',
 'loan_encoded',
 'contact_encoded',
 'month_encoded',
 'poutcome_encoded']



Primera aproximación para crear un modelo: crear obligatorio VectorAssembler, dividir los datos en train y test y entrenar la primera versión con hipermarámetros por defecto.

Y evaluamos nuestro modelo (usaremos accuracy)

In [13]:
# Respuesta

vector_assembler = VectorAssembler(inputCols=columns_for_model, outputCol='assembled_features')
bank_many_steps = vector_assembler.transform(bank_many_steps)

bank_many_steps_train, bank_many_steps_test = bank_many_steps.randomSplit([0.8,0.2])

rf = RandomForestClassifier(featuresCol=vector_assembler.getOutputCol(), labelCol=target_column)
rf_model = rf.fit(bank_many_steps_train)
bank_many_steps_prediction = rf_model.transform(bank_many_steps_test)

In [14]:
# Respuesta

evaluator = MulticlassClassificationEvaluator(predictionCol=rf.getPredictionCol(),
                                              labelCol=rf.getLabelCol(),
                                             metricName="accuracy")

In [15]:
# Respuesta

evaluator.evaluate(bank_many_steps_prediction)

0.8928176795580111



Podemos ver el número de árboles de nuestro random forest usando el atributo getNumTrees

In [16]:
# Respuesta

rf_model.getNumTrees

20



Pero, ¿podemos afirmar que este modelo fue entrenado con los mejores hiperparámetros? Se podrían listar muchas combinaciones de ellos, lo que significa mucho trabajo manual. Pero no hay que preocuparse, para manejar esta situación Spark incorpora **ParamGrid**. 

Sólo hay que definir los valores de cada hiperparámetro que queremos probar en ParamGridBuilder y utilizar el objeto resultante en el parámetro `estimatorParamMaps` cuando se define el CrossValidator.

En este caso, probaremos todas las combinaciones de:

- Number of trees: [10,50,100,200]
- Max depth: [3, 5, 7]
- Min instances per node: [3, 5, 7]

In [17]:
# Respuesta

grid = ParamGridBuilder().addGrid(rf.numTrees, [10,50,100,200]) \
                                .addGrid(rf.maxDepth, [3,5,7]) \
                                .addGrid(rf.minInstancesPerNode, [3,5,7]) \
                                .build()
rf_cv = CrossValidator(estimator=rf, 
                       estimatorParamMaps=grid, 
                       evaluator=evaluator, 
                       numFolds=3)
rf_cv_model = rf_cv.fit(bank_many_steps_test)

# In the attribute bestModel we have the best model after trying all the possible combinations of 
# hyperparameter values in a random forest, using accuracy as our metric and doing cross validation with 3 folds
bestModel = rf_cv_model.bestModel

In [19]:
# Respuesta

bestModel.getNumTrees

50

In [20]:
# Respuesta

evaluator.evaluate(bank_many_steps_prediction)

0.8928176795580111



Podemos encontrar todos los parámetros en el JavaObject del objeto modelo de pyspark

In [21]:
# Respuesta

bestModel._java_obj.getMinInstancesPerNode()

7